In [4]:
import os
from time import time
from pathlib import Path
import pandas as pd 
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import Dataset
import accelerate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [5]:
# Obtenemos la carpeta actual
current_dir = Path.cwd()
DATASETS_LOCATION = os.path.join(current_dir.parent.parent.parent, 'datasets')
MODELS_LOCATION = os.path.join(current_dir.parent.parent, 'models')

Cargamos el dataset:

In [6]:
df_disease_sp = pd.read_excel(os.path.join(DATASETS_LOCATION, 'disease_nlp_eng.xlsx'))

# Mostrar el DataFrame
df_disease_sp.head(3)

,Symptoms,Disease
0,I've been feeling tired all the time and notic...,Anemia
1,"Lately, I’ve been out of breath even after lig...",Anemia
2,"For the past few weeks, I’ve been extremely we...",Anemia


In [7]:
print("Dataset Disease English antes de undersampling:")
print(df_disease_sp['Disease'].value_counts())

Dataset Disease English antes de undersampling:
Disease
Anemia         100
Thalassemia    100
Thrombosis     100
Diabetes       100
Name: count, dtype: int64


Observamos que ya las clases están bien distribuidas. Dividimos el dataset en conjunto de entrenamiento y prueba:

In [8]:
# Dividiamo il dataset bilanciato in train (80%) e test (20%)
train_df, test_df = train_test_split(df_disease_sp, test_size=0.2, random_state=42, stratify=df_disease_sp['Disease'])

# Dividiamo ulteriormente il train in train (80%) e validation (20%)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['Disease'])

# Verifichiamo che i set siano equilibrati
print("\nConjunto de entrenamiento:")
print(train_df['Disease'].value_counts())

print("\nConjunto de validación:")
print(val_df['Disease'].value_counts())

print("\nConjunto de prueba:")
print(test_df['Disease'].value_counts())


Conjunto de entrenamiento:
Disease
Thrombosis     64
Anemia         64
Thalassemia    64
Diabetes       64
Name: count, dtype: int64

Conjunto de validación:
Disease
Thrombosis     16
Thalassemia    16
Anemia         16
Diabetes       16
Name: count, dtype: int64

Conjunto de prueba:
Disease
Thalassemia    20
Diabetes       20
Anemia         20
Thrombosis     20
Name: count, dtype: int64


In [9]:
# Crear un codificador de etiquetas para convertir las clases de texto en números
label_encoder = LabelEncoder()

# Convertir las clases de 'Disease' a números
train_df['label'] = label_encoder.fit_transform(train_df['Disease'])
val_df['label'] = label_encoder.transform(val_df['Disease'])
test_df['label'] = label_encoder.transform(test_df['Disease'])

train_df.head()

,Symptoms,Disease,label
281,"I’ve noticed my leg is swollen, and the veins ...",Thrombosis,3
22,"I’ve been getting infections a lot, and my ski...",Anemia,0
146,"I’ve been so weak lately, and my skin looks ye...",Thalassemia,2
70,I’ve noticed that the whites of my eyes have a...,Anemia,0
376,"My vision’s been blurry, I’m constantly thirst...",Diabetes,1


## Entrenamiento

In [10]:
# Definimos la función de métricas
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds, average='macro'),
        "recall": recall_score(p.label_ids, preds, average='macro'),
        "f1": f1_score(p.label_ids, preds, average='macro'),
    }

In [11]:
# Definimos el modelo y el tokenizer
model_checkpoint = "distilbert-base-uncased"  
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=4)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maria\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Preprocesamos los datos
def preprocess_function(examples):
    return tokenizer(examples['Symptoms'], examples['Disease'], padding=True, truncation=True, return_tensors="pt")

In [13]:
# Converti i DataFrame in dataset di Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(val_df)
# Tokenizamos el dataset
train_preprocessed_dataset = train_dataset.map(preprocess_function, batched=True)
test_preprocessed_dataset = test_dataset.map(preprocess_function, batched=True)
val_preprocessed_dataset = validation_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,              # Tasa de aprendizaje: se ha probado 5e-6, 1e-5, 5e-5
    per_device_train_batch_size=32,   # Tamaño del batch para entrenamiento: se ha probado 8, 16, 32, 64
    per_device_eval_batch_size=32,    # Tamaño del batch para evaluación
    num_train_epochs=3,
    weight_decay=0.01,               # Decaimiento de peso: hemos probado 0.01, 0.1, 0.2 e 0.3 e 0.4 y este era el mejor
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_preprocessed_dataset,
    eval_dataset=val_preprocessed_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics   # Función para calcular las métricas
)

c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\maria\AppData\Local\Temp\ipykernel_25044\2732148159.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Training

In [15]:
train_preprocessed_dataset

Dataset({
    features: ['Symptoms', 'Disease', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 256
})

In [16]:
# No modificar esta celda
# Esta celda, celda tiene que estar ejecutada en la entrega

start = time()

trainer.train()

end = time()
print(f">>>>>>>>>>>>> elapsed time: {(end-start)/60:.0f}m")

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.3688021898269653, 'eval_accuracy': 0.296875, 'eval_precision': 0.40350877192982454, 'eval_recall': 0.296875, 'eval_f1': 0.1844553512563182, 'eval_runtime': 1.0749, 'eval_samples_per_second': 59.54, 'eval_steps_per_second': 1.861, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.3432338237762451, 'eval_accuracy': 0.53125, 'eval_precision': 0.7159090909090909, 'eval_recall': 0.53125, 'eval_f1': 0.48641863278886877, 'eval_runtime': 1.2903, 'eval_samples_per_second': 49.601, 'eval_steps_per_second': 1.55, 'epoch': 2.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.3304280042648315, 'eval_accuracy': 0.578125, 'eval_precision': 0.8285714285714285, 'eval_recall': 0.578125, 'eval_f1': 0.5383406606988365, 'eval_runtime': 1.477, 'eval_samples_per_second': 43.331, 'eval_steps_per_second': 1.354, 'epoch': 3.0}
{'train_runtime': 66.5161, 'train_samples_per_second': 11.546, 'train_steps_per_second': 0.361, 'train_loss': 1.3562766710917156, 'epoch': 3.0}
>>>>>>>>>>>>> elapsed time: 1m


## Evaluación

In [17]:
# Evaluación del modelo en el conjunto de prueba

# Preprocesamos el conjunto de prueba
test_preprocessed_dataset = test_dataset.map(preprocess_function, batched=True)

# Evaluamos el modelo en el conjunto de prueba
results = trainer.evaluate(test_preprocessed_dataset)

# Imprimimos las métricas de evaluación
print("Resultados de la evaluación en el conjunto de prueba:")
print(f"Exactitud: {results['eval_accuracy']:.4f}")
print(f"Precisión: {results['eval_precision']:.4f}")
print(f"Recall: {results['eval_recall']:.4f}")
print(f"F1-Score: {results['eval_f1']:.4f}")

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Resultados de la evaluación en el conjunto de prueba:
Exactitud: 0.7000
Precisión: 0.8636
Recall: 0.7000
F1-Score: 0.6801


In [ ]:
# Guardamos el modelo y el tokenizer
save_directory = os.path.join(MODELS_LOCATION, 'disease_classification_spanish_nlp')
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Modelo y tokenizer guardados en el directorio {save_directory}")

In [ ]:
# Definamos algunas frases de ejemplo para el test que puedan confundir a nuestro modelo
sample_sentences = [
    {"Síntomas": "Me siento cansado todo el tiempo, como si no tuviera energía para nada.", "Enfermedad": "Anemia"},
    {"Síntomas": "Mi hijo no crece como debería y siempre se queja de estar débil.", "Enfermedad": "Talasemia"},
    {"Síntomas": "Tengo dolores en los huesos y me siento agotado incluso después de descansar.", "Enfermedad": "Talasemia"},
    {"Síntomas": "Tengo mucha sed todo el tiempo y no dejo de ir al baño.", "Enfermedad": "Diabetes"},
    {"Síntomas": "Tengo un dolor punzante y la pierna está hinchada y roja.", "Enfermedad": "Trombosis"},
    {"Síntomas": "He bajado de peso sin razón y me siento muy cansado.", "Enfermedad": "Diabetes"}
]

# Convertiamo le frasi di esempio in un DataFrame
sample_df = pd.DataFrame(sample_sentences)

# Tokenizziamo le frasi di esempio
sample_dataset = Dataset.from_pandas(sample_df)
sample_preprocessed_dataset = sample_dataset.map(preprocess_function, batched=True)

# Effettuiamo le predizioni
sample_predictions = trainer.predict(sample_preprocessed_dataset)
sample_preds = np.argmax(sample_predictions.predictions, axis=1)
sample_df["Enfermedad predicha"] = label_encoder.inverse_transform(sample_preds)

# Mostriamo i risultati
print("Resultados de las predicciones en las frases de ejemplo:")
sample_df